
<span style="color:#66D9EF; font-size:28px;">Gender Classification</span><br>


In [2]:
!pip install flask-cors



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Program Files\Python311\python.exe -m pip install --upgrade pip


In [10]:
import os
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import shutil
import os
print(os.getcwd())
import torch.nn as nn
import torch
import torchvision
import torchaudio
import torchvision.models as models
print(torch.__version__)
print(torchvision.__version__)
print(torchaudio.__version__)

import torch



c:\Users\215556754\Desktop\Gender
2.7.0+cpu
0.22.0+cpu
2.7.0+cpu


In [11]:
base_dir = "./Gender"

In [12]:
def plot_images(images, labels, sp=3):
    fig, axes = plt.subplots(sp, sp, figsize=(10, 10))
    fig.subplots_adjust(hspace=1, wspace=0.3)

    for ax, img_path, label in zip(axes.flatten(), images, labels):
        img = mpimg.imread(img_path)
        ax.imshow(img)
        ax.set_xlabel(f'Label: {label}', fontsize=12)
        ax.set_title(f'Label: {label}', fontsize=14)
        ax.axis("off")
    plt.tight_layout()
    plt.show()

Resize image with white background to maintain proportions

In [13]:
# שינוי גודל תמונה עם רקע לבן לשמירה על פרופורציות
def resize_with_white_background(path_ori, path_dest):
    img = Image.open(path_ori)
    
    # שינוי גודל תוך שמירה על יחס גובה-רוחב
    img.thumbnail((224, 224), Image.LANCZOS)
    
    # יצירת תמונה חדשה עם רקע לבן
    background = Image.new('RGB', (224, 224), (255, 255, 255))
    img_w, img_h = img.size
    offset = ((224 - img_w) // 2, (224 - img_h) // 2)
    background.paste(img, offset)
    
    # שמירת הקובץ בתקיית היעד
    background.save(os.path.join("resized", path_dest))


In [14]:
# import face_recognition
# from PIL import Image

# def crop_face_and_resize(path_ori, path_dest):
#     img = face_recognition.load_image_file(path_ori)
#     face_locations = face_recognition.face_locations(img)

#     if face_locations:
#         top, right, bottom, left = face_locations[0]  # ניקח רק את הפנים הראשונות
#         face_image = img[top:bottom, left:right]
#         pil_image = Image.fromarray(face_image)
#     else:
#         pil_image = Image.open(path_ori).convert("RGB")  # fallback אם אין פנים

#     # שינוי גודל תוך שמירה על יחס גובה-רוחב + רקע לבן
#     pil_image.thumbnail((224, 224), Image.LANCZOS)
#     background = Image.new('RGB', (224, 224), (255, 255, 255))
#     img_w, img_h = pil_image.size
#     offset = ((224 - img_w) // 2, (224 - img_h) // 2)
#     background.paste(pil_image, offset)
    
#     background.save(os.path.join("resized", path_dest))


In [15]:
# ריצה על כל התמונות בתיקייה ושינוי גודלן
# for img_path in tqdm(image_paths, desc="Resizing Images"):
#     img_name = os.path.basename(img_path)  # חילוץ שם הקובץ
#     resize_with_white_background(img_path, img_name)


In [16]:
#נתיה התיקייה בה התמונות לאחר שינוי הגודל
resized_dir = "./resized"


In [17]:
# איסוף כל התמונות מהתיקייה המעורבת
image_paths_resized = []
image_labels_resized  = []

for file in os.listdir(resized_dir):
    if file.lower().endswith((".jpg", ".jpeg", ".png")):
        # זיהוי מגדר מתוך שם הקובץ
        label = "male" if file.startswith("1") else "female"
        image_paths_resized .append(os.path.join(resized_dir, file))
        image_labels_resized .append(label)

# לבחור 9 תמונות רנדומליות
sp = 3  # 3x3 תמונות
num_images = min(sp * sp, len(image_paths_resized ))  # לוודא שאין יותר מ-9 תמונות
random_indices = np.random.choice(len(image_paths_resized ), num_images, replace=False)
selected_paths = [image_paths_resized [i] for i in random_indices]
selected_labels = [image_labels_resized [i] for i in random_indices]

# קריאה לפונקציה להצגת התמונות
# plot_images(selected_paths, selected_labels, sp)

In [18]:
# confirm the size of the resized images
im = Image.open(image_paths_resized[0])
resized_size = im.size
print(f"The size of the resized images: {resized_size}")


The size of the resized images: (224, 224)


In [19]:
# Variables to store paths based on gender
male_images = []
female_images = []

# Iterate over all files in the directory
for file in os.listdir(resized_dir):
    if file.lower().endswith((".jpg", ".jpeg", ".png")):
        file_path = os.path.join(resized_dir, file)

        # Classify based on the beginning of the file name
        if file.startswith("1"):
            male_images.append(file_path)  # Male
        elif file.startswith("0"):
            female_images.append(file_path)  # Female

# Print the number of files in each category
print(f"Number of male images: {len(male_images)}")
print(f"Number of female images: {len(female_images)}")


Number of male images: 2563
Number of female images: 2528


split to train, validate and test folders with the help of sklearn
each split is to 2 groups
so we need 2 splits in order to split to 3 groups

In [20]:
# the first split is to female_test and the rest of the images
female_train_valid, female_test = train_test_split(female_images, test_size=0.1, random_state=1)

# how many female in the test set?
print(f"female_test: {len(female_test)}")

# the second split on the female_train_valid folder separates to 2 folders: train and valid
female_train, female_valid = train_test_split(female_train_valid, test_size=0.2, random_state=1)

# how many female in the train and valid?
print(f"female_train: {len(female_train)}")
print(f"female_valid: {len(female_valid)}")


female_test: 253
female_train: 1820
female_valid: 455


In [21]:
# split the male dataset to the same ratio
male_train_valid, male_test = train_test_split(male_images, test_size=0.1, random_state=1)

male_train, male_valid = train_test_split(male_train_valid, test_size=0.2, random_state=1)
print(f"male_test: {len(male_test)}")
print(f"male_train: {len(male_train)}")
print(f"male_valid: {len(male_valid)}")


male_test: 257
male_train: 1844
male_valid: 462


In [22]:
male_test

['./resized\\1(1802).jpg',
 './resized\\1(1526).jpg',
 './resized\\1(1633).jpg',
 './resized\\1(937).jpg',
 './resized\\1(1328).jpg',
 './resized\\1(1250).jpg',
 './resized\\1(1528).jpg',
 './resized\\1(2561).jpg',
 './resized\\1(247).jpg',
 './resized\\1(897).jpg',
 './resized\\1(2025).jpg',
 './resized\\1(1505).jpg',
 './resized\\1(749).jpg',
 './resized\\1(1181).jpg',
 './resized\\1(354).jpg',
 './resized\\1(1593).jpg',
 './resized\\1(1266).jpg',
 './resized\\1(1003).jpg',
 './resized\\1(2505).jpg',
 './resized\\1(1823).jpg',
 './resized\\1(1416).jpg',
 './resized\\1(564).jpg',
 './resized\\1(1001).jpg',
 './resized\\1(803).jpg',
 './resized\\1(1272).jpg',
 './resized\\1(2408).jpg',
 './resized\\1(1909).jpg',
 './resized\\1(2110).jpg',
 './resized\\1(1281).jpg',
 './resized\\1(2151).jpg',
 './resized\\1(155).jpg',
 './resized\\1(1545).jpg',
 './resized\\1(1605).jpg',
 './resized\\1(1830).jpg',
 './resized\\1(39).jpg',
 './resized\\1(1090).jpg',
 './resized\\1(2011).jpg',
 './resized

In [23]:
#DataFrame נמיר את הרשימות ל   

female_train = pd.DataFrame(female_train, columns=['filename'])
female_test = pd.DataFrame(female_test, columns=['filename'])
female_valid = pd.DataFrame(female_valid, columns=['filename'])
male_train = pd.DataFrame(male_train, columns=['filename'])
male_test = pd.DataFrame(male_test, columns=['filename'])
male_valid = pd.DataFrame(male_valid, columns=['filename'])


In [24]:
# create the dataset by distributing the images to 3 folders: train, validate, test
# each containing 2 subfolders: female and male
for item in female_train['filename']:
  source = os.path.join("resized", os.path.basename(item))
  dest= 'dataset/train/female/' + os.path.basename(item)
  shutil.copy(source, dest)

for item in female_test['filename']:
  source = os.path.join("resized", os.path.basename(item))
  dest= 'dataset/test/female/' + os.path.basename(item)
  shutil.copy(source, dest)

for item in female_valid['filename']:
  source = os.path.join("resized", os.path.basename(item))
  dest= 'dataset/validate/female/' + os.path.basename(item)
  shutil.copy(source, dest)


for item in male_train['filename']:
  source = os.path.join("resized", os.path.basename(item))
  dest= 'dataset/train/male/' + os.path.basename(item)
  shutil.copy(source, dest)

for item in male_test['filename']:
  source = os.path.join("resized", os.path.basename(item))
  dest= 'dataset/test/male/' + os.path.basename(item)
  shutil.copy(source, dest)

for item in male_valid['filename']:
  source = os.path.join("resized", os.path.basename(item))
  dest= 'dataset/validate/male/' + os.path.basename(item)
  shutil.copy(source, dest)


In [25]:
# paths to the datasets
train_path = 'dataset/train/'
valid_path = 'dataset/validate/'
test_path  = 'dataset/test/'

In [26]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# הגדרת transforms
train_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
])
val_test_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
])

train_dataset = datasets.ImageFolder('dataset/train', transform=train_transforms)
val_dataset   = datasets.ImageFolder('dataset/validate', transform=val_test_transforms)
test_dataset  = datasets.ImageFolder('dataset/test',  transform=val_test_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=32, shuffle=False)
test_loader  = DataLoader(test_dataset,  batch_size=32, shuffle=False)


In [27]:
images, labels = next(iter(train_loader))
print(images.shape)  # צפוי: torch.Size([32, 3, 224, 224])
print(labels.shape)  # צפוי: torch.Size([32])


torch.Size([32, 3, 224, 224])
torch.Size([32])


In [28]:
import torch.nn as nn
from torchvision import models

model = models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 128),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(128, 1)
)


In [29]:
import torch, torch.nn as nn, torch.optim as optim

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=1e-4)

for epoch in range(100):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs.squeeze(), labels.float())
        loss.backward()
        optimizer.step()
    # eval על val_loader...


In [32]:
model.eval()
total, correct = 0, 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        preds = (torch.sigmoid(outputs).squeeze() > 0.5).int()
        total += labels.size(0)
        correct += (preds == labels).sum().item()
print(f"Test Accuracy: {correct/total:.4f}")


Test Accuracy: 0.9431


In [31]:
torch.save(model.state_dict(), "my_model.pt")
# model.save('my_model.h5')  # או לתיקייה: model.save('my_model')